# **Введение**

В этом модуле вы попробуете решить свой первый настоящий кейс и создадите первую модель, использующую алгоритмы машинного обучения.

**[Представьте, что вы работаете дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.]**

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import display
import logging
from comet_ml import Experiment

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки для обучения модели:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

# **Разбираемся с данными**

В этом модуле вы будете работать с датасетом, в котором содержатся сведения о 515 000 отзывов на отели Европы. Модель, которую вы будете обучать, должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных. Изученные нами навыки разведывательного анализа помогут улучшить модель. Как? Узнаем дальше.

1. **hotel_address** — адрес отеля;
2. **review_date** — дата, когда рецензент разместил соответствующий отзыв;
3. **average_score** — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
4. **hotel_name** — название отеля;
5. **reviewer_nationality** — страна рецензента;
6. **negative_review** — отрицательный отзыв, который рецензент дал отелю;
7. **review_total_negative_word_counts** — общее количество слов в отрицательном отзыв;
8. **positive_review** — положительный отзыв, который рецензент дал отелю;
9. **review_total_positive_word_counts** — общее количество слов в положительном отзыве.
10. **reviewer_score** — оценка, которую рецензент поставил отелю на основе своего опыта;
11. **total_number_of_reviews_reviewer_has_given** — количество отзывов, которые рецензенты дали в прошлом;
12. **total_number_of_reviews** — общее количество действительных отзывов об отеле;
13. **tags** — теги, которые рецензент дал отелю;
14. **days_since_review** — количество дней между датой проверки и датой очистки;
15. **additional_number_of_scoring** — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
16. **lat** — географическая широта отеля;
17. **lng** — географическая долгота отеля.

In [2]:
hotels = pd.read_csv('data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [3]:
# В каких столбцах данные содержат пропущенные значения?
# В каких столбцах данные хранятся в числовом формате?
# В каких столбцах данные представляют собой строки?
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [4]:
# чистим датасет по тупому

hotels_dumb_prep = hotels.select_dtypes(include=['int64','float64'])

hotels_dumb_prep.fillna(value={'lat': hotels_dumb_prep['lat'].mean(), 'lng': hotels_dumb_prep['lng'].mean()}, inplace=True)
hotels_dumb_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 9 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                386803 non-null  int64  
 1   average_score                               386803 non-null  float64
 2   review_total_negative_word_counts           386803 non-null  int64  
 3   total_number_of_reviews                     386803 non-null  int64  
 4   review_total_positive_word_counts           386803 non-null  int64  
 5   total_number_of_reviews_reviewer_has_given  386803 non-null  int64  
 6   reviewer_score                              386803 non-null  float64
 7   lat                                         386803 non-null  float64
 8   lng                                         386803 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 26.6 MB


c:\Program Files\Python39\lib\site-packages\pandas\core\series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


# **Машинное обучение для самых маленьких**

✍ Что нужно сделать в этом юните?

Создать свою первую модель, основанную на алгоритмах машинного обучения.

Код для создания модели мы предоставим, но чтобы модель заработала, в неё нужно передать правильные данные. Приведение данных в нужный вид и будет вашей задачей в этом юните.

Однако прежде мы рассмотрим основные шаги, которые нужно проделать с данными от загрузки до получения результатов модели.  
Мы подготовили для вас ноутбук с кодом для обучения модели. Далее мы шаг за шагом разберём все описанные в нём действия.

## **1. РАЗДЕЛЕНИЕ НАБОРА ДАННЫХ**

Прежде всего, для создания модели необходимо разделить датафрейм на набор данных, которые мы будем использовать для обучения модели, именуемый **X**, и на целевую переменную — величину, значение которой мы будем предсказывать, **y** (в нашем случае это рейтинг отелей).

![](https://lms.skillfactory.ru/assets/courseware/v1/2962190377a2a4c55ae3f6b5643f1ccc/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-pr3_3_1.png)

Далее каждый из полученных наборов мы делим на тренировочный (train, используется для обучения модели) и тестовый (test, используется для оценки точности модели). Такое деление осуществляется с помощью специального метода train_test_split() библиотеки sklearn. В параметрах метода (параметр test_size) мы указываем, какую часть исходного датафрейма нужно оставить для тестирования модели. В нашем коде эта часть составляет 25 %, или 0.25.

*Не существует идеального соотношения обучающей и тестовой выборки, однако принято выделять на часть обучения больше данных, около ⅔, а оставшуюся часть используют для проверки — ⅓ выборки. Таким образом, самые популярные значения параметра test_size составляют 0.3, 0.25, 0.2.*



In [5]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х — данные с информацией об отелях, у — целевая переменная (рейтинги отелей)  
X = hotels_dumb_prep.drop(['reviewer_score'], axis = 1)
y = hotels_dumb_prep['reviewer_score']

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" — для тестирования.  
# Для тестирования мы будем использовать 25 % от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## **2. СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ**

Сам процесс создания и тестирования модели занимает всего четыре строчки кода. В качестве алгоритма мы будем использовать популярный и довольно мощный алгоритм **RandomForestRegressor**. Он реализован в библиотеке sklearn.

In [6]:
# СНАЧАЛА УБРАТЬ ВСЕ НЕЧИСЛОВЫЕ ПРИЗНАКИ


# Создаём модель
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

## **3. ОЦЕНКА КАЧЕСТВА МОДЕЛИ** 

Для оценки качества модели — точности прогнозов, сделанных моделью, — мы будем использовать метрику (некий числовой показатель), которая называется **MAPE (mean absolute percentage error)**, средняя абсолютная процентная ошибка. Эту метрику очень легко интерпретировать. Если у вас MAPE = 11.4 %, то это говорит о том, что ошибка составила 11.4 % от фактических значений.

In [7]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test) и смотрим, насколько они отличаются  

# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную
# процентную ошибку предсказанных значений в отношении фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.1412715398193932


Итак, мы выяснили, что для корректной работы все данные в датафрейме, который вы будете использовать при обучении модели, должны быть в числовых форматах int или float.
Есть ещё одно ограничение: в столбцах не должно быть пропущенных значений. Вместо каждого пропущенного значения вам нужно будет вычислить и поместить в ячейку максимально близкое к реальности значение.


**Таким образом, чтобы код, отвечающий за обучение модели, заработал, мы должны использовать в нём датафрейм, содержащий только количественные признаки и не содержащий пропущенных значений.**

На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие данные типов object и string и заполним пропущенные значения каким-то одним значением (unknown, нулём или средним арифметическим) для всего столбца.

Для выполнения этих операций вам понадобятся следующие методы библиотеки Pandas:

* **drop** — метод для удаления ненужных строк и столбцов (обратите внимание, что в материалах данного модуля этот метод разрешается использовать только для удаления столбцов);
* **fillna** — метод для заполнения пропущенных значений в столбце или во всём датафрейме.
  

In [8]:
# Сколько отелей представлено в наборе данных?

hotels['hotel_name'].describe()

count                                         386803
unique                                          1492
top       Britannia International Hotel Canary Wharf
freq                                            3587
Name: hotel_name, dtype: object

In [9]:
# Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.

hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_date'] = pd.to_datetime(hotels['review_date'], format='%Y-%m-%d')
display(hotels['review_date'].min())
display(hotels['review_date'].max())

Timestamp('2015-08-04 00:00:00')

Timestamp('2017-08-03 00:00:00')

In [10]:
# Сколько уникальных тегов представлено в наборе данных?
regex = """\'\s(.*?)\s\'""" # регулярное выражение
hotels['tags'] = hotels['tags'].str.findall(regex)

In [19]:
hotels['tags']
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                     386803 non-null  int64        

In [15]:
# Какой тег представлен в наибольшем числе отзывов?
# Из тегов выясните, на сколько ночей чаще всего останавливаются путешественники в отелях.

# собираем количества упоминаний каждого тега в две переменных
unique, counts = np.unique(hotels['tags'].explode(), return_counts=True)
# составляем датафрейм из этих переменных
tag_freq = pd.DataFrame(data=np.asarray((unique, counts)).T, columns=['tag','frequency'])
# сортируем датафрейм по убыванию
tag_freq.sort_values(by=['frequency'], ascending=False, inplace=True)
display(tag_freq.head(30))

,tag,frequency
1237,Leisure trip,313593
1842,Submitted from a mobile device,230778
272,Couple,189212
1788,Stayed 1 night,145373
1799,Stayed 2 nights,100263
1642,Solo traveler,81235
1810,Stayed 3 nights,72000
105,Business trip,61989
983,Group,49088
943,Family with young children,45836
